In [1]:
import json
config = {'model': {'type': ['abnormal'], 'process_kpi': 3, 'process_event': 5},
             'kafka': {'http': '192.168.50.84:19092',
                  'event_topic': 'dc_event',
                  'offset': 'latest',
                  'groupid': 'apm-rca',
                  'rca_topic': 'dc_rca_test'},
             'mysql': {'host': '192.168.50.84',
                  'port': 13306,
                  'user': 'root',
                  'password': '234*(sdlj12',
                  'database': 'dc_databuff'},
            'influxDB': {'host': '192.168.50.84',
                  'port': 18086,
                  'user': 'admin',
                  'password': '234*(sdlj12',
                  'dbname': 'source_metric',
                  'table': 'NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4'},
             'elasticsearch': {'http': '192.168.50.84:19200',
                  'span_index': 'dc_span',
                  'kpi_index': 'dc_source_metrics',
                  'rca_index': 'dc_rca'}}
# config

## ElasticSearch 取数

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(config['elasticsearch']['http'])

In [3]:
def scroll_search(body, index):
    batches = 100  # 每一批数据量大小
    queryData = es.search(index=index, scroll='5m', timeout='60s', size=batches, body=body)
    metadata = queryData.get("hits").get("hits")
    scroll_id = queryData["_scroll_id"]
    total = queryData["hits"]["total"]['value']
    for i in range(int(total / batches)):
        res = es.scroll(scroll_id=scroll_id, scroll='5m')  # scroll参数必须指定否则会报错
        metadata += res["hits"]["hits"]
    es.clear_scroll(scroll_id=scroll_id)
    return metadata

In [4]:
import time
t = int(time.time())
body = {
    'query': {
        'bool': {
            'must': [{'term': {"service_id": 'TnpoRU1qbEJPVGszTXpreFJqazVNalF5TXpNelF6STR8b3JkZXItY291cG9uLXNlcnZpY2U6OnNlcnZpY2Vbb3JkZXIxNF0='}},
                    {'range': {'startTime': {'gte': t*1000-600000, 'lt': t*1000}}} # 左闭右开
                    ]
        }
    }
}
data = scroll_search(body, config['elasticsearch']['span_index']+'*')
# self.monitor_service = set([l['_source']['service_id'] for l in data])
# print("监控的service_id有：{}".format(self.monitor_service))

In [5]:
data[0]

{'_index': 'dc_span_20220929',
 '_type': '_doc',
 '_id': 'Sh1riIMBvvhuI0j81qrw',
 '_score': 4.771347,
 '_source': {'process_id': 'ab9e56b0d0038ef6d89a81f7b1de4285',
  'hostName': 'order14',
  'ports': '8093,8690',
  'error': 0,
  'type': 'web',
  'original_service': 'order-coupon-service::service',
  'duration': 13530671,
  'service_id': 'TnpoRU1qbEJPVGszTXpreFJqazVNalF5TXpNelF6STR8b3JkZXItY291cG9uLXNlcnZpY2U6OnNlcnZpY2Vbb3JkZXIxNF0=',
  'startTime': 1664441105587,
  'x-dd-hostname': 'order14',
  'timestamp': 1664441110,
  'trace_id': '68880306564973699',
  'resource': 'POST /org.databuff.apm.order.api.api.coupon.OrderCouponService',
  'span_id': '3398877772307448089',
  'dd-api-key': 'NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4',
  'start': 1664441105587008077,
  'host_id': 'e187a093b80614853f145be8c2b15dc4',
  'data_source': 'local',
  'call': 1,
  'service': 'order-coupon-service::service[order14]',
  'parent_id': '4745471804142444135',
  'meta': {'http.status_code': '200',
   'component': 'je

## Mysql 取数

In [6]:
import pymysql

In [7]:
mysql_connection = pymysql.connect(**config['mysql']) # 第一步：创建数据库连接对象
cur = mysql_connection.cursor(cursor=pymysql.cursors.DictCursor) # 第二步：创建游标，默认是返回元组数据类型，加上cursor参数，指定返回字典
cur.execute('select * from dc_databuff_monitor') # 第三步：执行数据库操作
# mysql_connection.commit() # 如果是增产改操作的，需要进行提交操作,通过数据库连接对象来调用 
data = cur.fetchall() # 第四步：查看数据库操作结果
cur.close() # 第五步: 关闭游标
mysql_connection.close() # 第六步：关闭数据库连接

In [8]:
data[0]

{'id': 265,
 'name': 'order-mysql错误率大于0',
 'tags': '[]',
 'classification': 'apm',
 'type': 'threshold',
 'priority': 0,
 'multi': 0,
 'time_out': 0,
 'message': 'order-mysql错误率大于0',
 'query': '{"type":"metric","metric":"errRate","service":"mysql[Order16]","serviceId":"TnpoRU1qbEJPVGszTXpreFJqazVNalF5TXpNelF6STR8bXlzcWxbT3JkZXIxNl0=","resource":["*"]}',
 'options': '{"thresholds":{"critical":0,"warning":"","unknown":"","nodata":""},"timeout_h":0,"new_group_delay":0,"unit":"fractions"}',
 'params': '{"period":60,"metric":"errRate","comparison":">"}',
 'monitor_notice': None,
 'create_time': datetime.datetime(2022, 9, 20, 9, 50, 35),
 'update_time': datetime.datetime(2022, 9, 20, 9, 50, 35),
 'api_key': 'NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4'}

## InfluxDB取数

In [9]:
from influxdb import InfluxDBClient
import pandas as pd

In [10]:
client = InfluxDBClient(host = config['influxDB']['host'], # new
    port = config['influxDB']['port'],
    username = config['influxDB']['user'], # new
    password = config['influxDB']['password'],
    database = config['influxDB']['dbname'])
table = config['influxDB']['table'] # new

In [11]:
data = client.query("select * from {} where metric ='{}' limit 10".format(table,'system.swap.total'))
pd.DataFrame(data[table])

,time,J2EEApplication,J2EEServer,WebModule,action,agent_version_major,agent_version_minor,agent_version_patch,api,apiKey,...,tracer_version,type,uid,upper_bound,url,username,v,verb,version,volume_type
0,2022-09-14T00:00:12Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,9496581fd3414be0a71ead83be8223bb,None,None,None,None,None,None,None
1,2022-09-14T00:00:23Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,d58af7bf3344e97e937b29331db07286,None,None,None,None,None,None,None
2,2022-09-14T00:00:25Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,5da91265791394e03a4a4c116c0fecf4,None,None,None,None,None,None,None
3,2022-09-14T00:00:27Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,a2c4d5e389a68968d59fecd393a684d9,None,None,None,None,None,None,None
4,2022-09-14T00:00:30Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,19a85da348665fdbfcb7b360d6e75733,None,None,None,None,None,None,None
5,2022-09-14T00:00:31Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,b4002950b65030e4d1b9f713a9354825,None,None,None,None,None,None,None
6,2022-09-14T00:00:32Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,e25c4b93ba314d19b41f7398b103f0ba,None,None,None,None,None,None,None
7,2022-09-14T00:00:38Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,449be3d4616897028275d0acb07ef3f5,None,None,None,None,None,None,None
8,2022-09-14T00:00:39Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,2349c039b1a96796512b368633fa8693,None,None,None,None,None,None,None
9,2022-09-14T00:00:41Z,None,None,None,None,None,None,None,None,NzhEMjlBOTk3MzkxRjk5MjQyMzMzQzI4,...,None,None,433b5abe3621fdd464d433b23019f035,None,None,None,None,None,None,None
